## **Mount Drive and Set path**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [32]:
dir_monhoc = '/content/drive/MyDrive/CS231.M21.KHTN'
dir_baitap = os.path.join(dir_monhoc, 'Baitap', 'Baitap3')
dir_train = os.path.join(dir_monhoc,'Baitap', 'Dataset','dogscats_small','train')
dir_test = os.path.join(dir_monhoc,'Baitap', 'Dataset','dogscats_small','test')

## **Installation Function**

In [5]:
def TinhHist(pathfilename):
  img = cv2.imread(pathfilename,0)
  hist = cv2.calcHist([img],[0],None,
  [256],[0,256])
  size = img.shape[0]*img.shape[1]
  hist = hist / size
  return hist

In [63]:
from skimage import feature

def compute_hog_train(pathfilename):
  train_features =[]
  #for img in train_images:
  img = cv2.imread(pathfilename)
  img = cv2.resize(img, (128, 256))
  (hog, hog_image) = feature.hog(img, orientations=9,
      pixels_per_cell=(8, 8), cells_per_block=(2, 2),
      block_norm='L2-Hys', visualize=True, transform_sqrt=True)

  train_features.append(hog)
  return train_features

In [68]:
def ExtractFeatures(pathfilename, opt):
  if opt==0:
    return TinhHist(pathfilename)
  if opt==1:
    return compute_hog_train(pathfilename)[0].reshape(-1)

In [69]:
def ReadData(path,opt):
  kinds = []
  hist = []
  for file_name in os.listdir(path):
    kind = file_name.split('.')[0]
    if kind == 'dog':
      kinds.append(1)
    else:
      kinds.append(0)
    hist.append(ExtractFeatures(path + '/' + file_name, opt).reshape(-1))
    
    features = np.array(hist)
    labels = np.array(kinds)
  return features, labels

In [62]:
compute_hog_train('/content/drive/MyDrive/CS231.M21.KHTN/Baitap/Dataset/dogscats_small/train/cat.1.jpg')[0]

array([0.25203475, 0.01100299, 0.04007513, ..., 0.16720066, 0.18470497,
       0.23205595])

## **Linear classification on Histogram**

In [54]:
X_train , y_train = ReadData(dir_train,0)
X_test, y_test = ReadData(dir_test,0)

In [55]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [56]:
predicted_lables = clf.predict(X_test)
print(f'predicted acccuracy : {np.mean(predicted_lables == y_test)}')

predicted acccuracy : 0.45


## **KNN and Linear classification on HOG**

# **KNN**

In [81]:
X_train, y_train = ReadData(dir_train,1)
X_test, y_test = ReadData(dir_test,1)

In [71]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

In [84]:
for k in range(1,11):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train,y_train)
  predicted_labels = neigh.predict(X_test)
  print(f'K = {k}, Accuracy = { np.mean(predicted_labels == y_test) }')

K = 1, Accuracy = 0.65
K = 2, Accuracy = 0.65
K = 3, Accuracy = 0.55
K = 4, Accuracy = 0.6
K = 5, Accuracy = 0.55
K = 6, Accuracy = 0.55
K = 7, Accuracy = 0.45
K = 8, Accuracy = 0.45
K = 9, Accuracy = 0.4
K = 10, Accuracy = 0.4


# **Linear Classification**

In [78]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [79]:
predicted_lables = clf.predict(X_test)
print(f'predicted acccuracy : {np.mean(predicted_lables == y_test)}')

predicted acccuracy : 0.65
